In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [2]:
import os
import json
import shutil

In [16]:

# # Paths to the folder of first-stage JSONs and the second-stage JSON
# first_stage_results_folder = "../resnet_model/results/combine_first_and_second_stage_results/mask2former_test_triplet_segmentation_v2_dataset_on_full"
# second_stage_file = "../resnet_model/results/verb_target_prediction/predict_mask2former_test_triplet_segmentation_v2_results_full.json"

In [3]:
first_stage_results_source_folder = "../data/triplet_segmentation_dataset_v2_second_stage/mask2former_instrument_prediction/ann_dir"
first_stage_results_destination_folder = "../resnet_model/work_dirs/multitask_resnet_standard/combine_first_and_second_stage_results"
second_stage_results_file = "../resnet_model/work_dirs/multitask_resnet_standard/results.json"

In [4]:
shutil.copytree(first_stage_results_source_folder, first_stage_results_destination_folder)

'../resnet_model/work_dirs/multitask_resnet_standard/combine_first_and_second_stage_results'

In [5]:
from utils.general.dataset_variables import TripletSegmentationVariables

VERB_ID_TO_CLASS_DICT = TripletSegmentationVariables.categories['verb']
TARGET_ID_TO_CLASS_DICT = TripletSegmentationVariables.categories['target']

In [6]:
# Load the second-stage results
with open(second_stage_results_file, 'r') as f:
    second_stage_results = json.load(f)

# Iterate through all JSON files in the first-stage folder
for file_name in os.listdir(first_stage_results_destination_folder):
    file_name_no_ext = file_name.split('.')[0]
    if file_name.endswith(".json"):
        first_stage_path = os.path.join(first_stage_results_destination_folder, file_name)
        
        # Load the first-stage JSON
        with open(first_stage_path, 'r') as f:
            first_stage_data = json.load(f)

        # Iterate through each shape in the first-stage JSON
        for shape in first_stage_data['shapes']:
            # Construct the unique key used in the second-stage results
            
            image_key = f"{file_name_no_ext},{shape['label']},{shape['group_id']},{shape['verb']},{shape['target']}"
            
            # If the key exists in the second-stage results, update the shape
            if image_key in second_stage_results:
                second_stage_info = second_stage_results[image_key]
                shape['verb'] =  VERB_ID_TO_CLASS_DICT[str(second_stage_info['verb'] + 1)]
                shape['target'] = TARGET_ID_TO_CLASS_DICT[str(second_stage_info['target'] + 1)]
                shape['verb_score'] = second_stage_info.get('verb_score', None)
                shape['target_score'] = second_stage_info.get('target_score', None)
            else: 
                print(f'imagekey does not exist {image_key}')    

        # Save the updated JSON back to the folder
        with open(first_stage_path, 'w') as f:
            json.dump(first_stage_data, f, indent=4)

print("Updated all first-stage JSONs with second-stage results.")


Updated all first-stage JSONs with second-stage results.
